In [178]:
import os
import json
import yaml

In [179]:
course_2_path = os.path.expanduser('~') + '/Coursera/Course2/Networking Services'
course_2 = os.listdir(course_2_path)

In [180]:
question_list = [i for i in course_2 if '.json' in i and 'question' in i]
question_list = ['question_' + str(i) + '.json' for i in range(1, len(question_list) + 1)]

answer_file = [i for i in course_2 if '.json' in i and 'answer' in i]
question_list.append(answer_file[0])
file_list = question_list

In [181]:
# file_list

In [182]:
question_list = list()

for file in file_list:
    file_path = os.path.join(course_2_path, file)
    if 'question' in file_path:
        question_file = file_path
        with open(question_file) as file:
            question_data = json.loads(file.read())
            question_data['question_number'] = int(question_file.split('.')[0].split('_')[1])
            question_list.append(question_data)
            file.close()
    
    if 'answer' in file_path:
        answer_file = file_path
        with open(answer_file) as file:
            raw_answer_data = json.loads(file.read())
            file.close()

question_with_question_number_list = [[question['question_number'],question['questions']]  for question in question_list]

In [183]:
def find_answer(answer_id, raw_answer_data):
    if answer_id in raw_answer_data.keys():
        correct_answer_id = raw_answer_data[answer_id]['correctAnswer']
        question_option_id_list = list(raw_answer_data[answer_id]['feedbacks'].keys())
        true_data, false_data = dict(), dict()
#         print('---**', raw_answer_data[answer_id]['type'])
#         print('---**', len(raw_answer_data[answer_id]['feedbacks']))
        if type(correct_answer_id) == str and raw_answer_data[answer_id]['type'] == 'multipleChoice':
            true_rationale = raw_answer_data[answer_id]['feedbacks'][correct_answer_id]
            true_data[correct_answer_id] = true_rationale

            question_option_id_list.pop(question_option_id_list.index(correct_answer_id))
            
            for id in question_option_id_list:
                false_data[id] = raw_answer_data[answer_id]['feedbacks'][id]
            
            true_data.update(false_data)
            
            return true_data, correct_answer_id
        
        elif type(correct_answer_id) == list and raw_answer_data[answer_id]['type'] == 'checkbox':
            false_answer_id = list(set(question_option_id_list) - set(correct_answer_id))
            
            for ans_id in correct_answer_id:
                if ans_id in question_option_id_list:
                    true_data[ans_id] = raw_answer_data[answer_id]['feedbacks'][ans_id]
            
            for fal_id in false_answer_id:
                if fal_id in question_option_id_list:
                    false_data[fal_id] = raw_answer_data[answer_id]['feedbacks'][fal_id]
            
            true_data.update(false_data)
            
            return true_data, correct_answer_id
        else:
            pass
            

In [184]:
def get_questions(raw_data):
    total_questions = list()
    for _ in raw_data:
        question_number = _[0]
        question_data = _[1]
        question_list = list()
        for _ in question_data:
            question_d = {'answer_id': _['id'],
                          'question_number': question_number,
                          'question_options': _['options'],
                          'question': _['prompt'],
                          'type': _['type']}
            question_list.append(question_d)
        total_questions.append(question_list)
    return total_questions   

In [185]:
get_questions_list = get_questions(question_with_question_number_list)

In [186]:
get_data = list()

for get_questions in  get_questions_list:
    question_number = list(set([get_questions[i]['question_number'] for i in range(len(get_questions))]))[0]
    final_data_in_a_quiz = list()
    
    for _ in get_questions:
        data = dict()
        answer_id = _['answer_id']
        data["question"] = _['question']
        data["question_number"] = _['question_number']
        data['question_type'] = _['type']
        question_id_with_options = {option['id']: option['answer'] for option in _['question_options']}
        
        if question_id_with_options.keys() == find_answer(answer_id, raw_answer_data)[0].keys():
            option_answer = {question_id_with_options[i]: find_answer(answer_id, raw_answer_data)[0][i] for i in  question_id_with_options.keys()}
            data["optional_rationale"] = option_answer
            
            if type(find_answer(answer_id, raw_answer_data)[1]) == str:
                data['correct_answer'] = question_id_with_options[find_answer(answer_id, raw_answer_data)[1]]
            else:
                correct_ids = find_answer(answer_id, raw_answer_data)[1]
                data['correct_answer'] = [question_id_with_options[id] for id in correct_ids]
                
        final_data_in_a_quiz.append(data)
    get_data.append(final_data_in_a_quiz)

In [202]:
raw_json_data = {
    'entity_type': 'Quiz',
    'schema_version': 1,
    'title': {'locales': {'en': 'Graded Quiz - ' + course_2_path.split('/')[-1]}},
    'passing_percentage': 80,
    'duration': 50,
    'default_locale': 'en'
}
total_question_item = list()

for data in get_data:
    question_bank = list()
    question_bank_in_section = dict()
    question_bank_in_section['item_count'] = 1
    question_bank_in_section['items'] = list()
    
    for item_number, _ in enumerate(data):
        section_number = _['question_number']
        question_bank_in_section['id'] = "section-" + str(section_number)
        question_bank_in_section['name'] = question_bank_in_section['id'] + '-quiz'
        section_item = {'stem' : {'locales': {'en' :_['question']}}, 'options': list()}
        section_item['id'] = 'section-' + str(section_number) + '-item-' + str(item_number)
        
        for option_number, title in enumerate(_['optional_rationale']):
            my_dict = {}
            my_dict['id'] = "section-" + str(section_number) + "-item-" + str(item_number) + "-option-" + str(option_number)
            
            if _['question_type'] == 'multipleChoice':
                if len(_['optional_rationale']) == 4:
                    section_item['type'] = 'multiple-choice'
                    my_dict['title'] = dict()
                    my_dict['title']['locals'] = dict()
                    my_dict['title']['locals']['en'] = title
#                     print(type(_['correct_answer']), _, _['correct_answer'])
#                     print(title, type(title))
                    my_dict['is_answer'] = True if title == _['correct_answer'] else False
                    my_dict['rationale'] = dict()
                    my_dict['rationale']['locals'] = dict()
                    my_dict['rationale']['locals']['en'] = _['optional_rationale'][title]
                
                if len(_['optional_rationale']) == 2:
                    section_item['type'] = 'true-false'
                    my_dict['answer'] = True if title == _['correct_answer'] else False
                    my_keys = list(_['optional_rationale'].keys())
                    my_dict['id'] = "section-" + str(section_number) + "-item-" + str(item_number)
                    for rationale in my_keys:
                        if rationale.lower() == 'true':
                            my_dict['true_rationale']  = dict()
                            my_dict['true_rationale']['locals'] = dict()
                            my_dict['false_rationale']  = dict()
                            my_dict['false_rationale']['locals'] = dict()
                            
                            temp_list = list()
                            temp_list.append(rationale)
                            false_rationale = list(set(my_keys) - set(temp_list))[0]
                            
                            my_dict['true_rationale']['locals']['en'] = _['optional_rationale'][rationale]
                            my_dict['false_rationale']['locals']['en'] = _['optional_rationale'][false_rationale]
                            
                            my_dict['answer'] = True
                    
            elif _['question_type'] == 'checkbox':
                section_item['type'] = 'multiple-select'
                my_dict['title'] = dict()
                my_dict['title']['locals'] = dict()
                my_dict['title']['locals']['en'] = title
                my_dict['is_answer'] = True if title in _['correct_answer'] else False
                my_dict['rationale'] = dict()
                my_dict['rationale']['locals'] = dict()
                my_dict['rationale']['locals']['en'] = _['optional_rationale'][title]
            
            
            section_item['options'].append(my_dict)
        section_item['options'] = list({item['id']: item for item in section_item['options']}.values())
        question_bank.append(section_item)
        
    question_bank_in_section['items'] = question_bank
    total_question_item.append(question_bank_in_section)

    raw_json_data['sections'] = total_question_item

In [203]:
with open(os.path.join(course_2_path,'qwiklabs.json'), mode = 'w+') as file:
    raw_json = "{}".format(json.dumps(raw_json_data,sort_keys=True, indent = 4))
    file.write(raw_json)